In [1]:
import rasterstats
import rasterio

In [2]:
import shapely

DATA_POLYGON = shapely.Polygon(
    [[359326,4833160],
     [376735,4842547],
     [385238,4826271],
     [367914,4816946],
     [359326,4833160]]
)

In [3]:
#zonal_stats(
#    DATA_POLYGON, 
#    "/data/DIGITANIE/Toulouse/Toulouse.tif",
#    stats="count min mean max median"
#)

In [4]:
def generate_polygon(bbox):
    """
    Generates a list of coordinates: [[x1,y1],[x2,y2],[x3,y3],[x4,y4],[x1,y1]]
    """
    return [[bbox[0],bbox[1]],
             [bbox[2],bbox[1]],
             [bbox[2],bbox[3]],
             [bbox[0],bbox[3]],
             [bbox[0],bbox[1]]]

In [16]:
import numpy as np
from dl_toolbox.utils import get_tiles
with rasterio.open("/data/DIGITANIE/Toulouse/Toulouse.tif") as f:
    h,w = f.shape
    full_in = False
    n = 0
    while not full_in:
        crop = rasterio.windows.Window(n, n, h-2*n, w-2*n)                            
        left, bottom, right, top = rasterio.windows.bounds(
            crop,
            transform=f.transform
        )
        crop_poly = shapely.Polygon(generate_polygon((left, bottom, right, top)))
        inter = shapely.intersection(crop_poly, DATA_POLYGON)
        area = shapely.area(inter) / shapely.area(crop_poly)
        full_in = (area>=.99)
        print(full_in)
        n += 1000
    n -= 1000
    mean = np.array([0,0,0,0], dtype=np.float64)
    i = 1
    for tile in get_tiles(w-2*n, h-2*n, 5000, col_offset=n, row_offset=n, cover_all=False):
        tile_img = f.read(window=tile)
        tile_mean = np.mean(tile_img, axis=(1,2))
        mean += (tile_mean - mean) / i
        i += 1
        print(mean)
    print(mean)
    #print(crop_img.shape)

False
False
False
False
False
False
False
False
False
False
False
False
False
True
[0.09849199 0.10335866 0.11288217 0.21531706]
[0.10470457 0.10885551 0.11775978 0.21399323]
[0.10903376 0.11242765 0.12168191 0.20664583]
[0.10844705 0.11138627 0.12069001 0.20740956]
[0.10557562 0.10966521 0.11909904 0.20698276]
[0.10710744 0.11109315 0.12032198 0.20756214]
[0.11020168 0.11371848 0.1227758  0.2059096 ]
[0.11283427 0.11598756 0.12498699 0.20361528]
[0.11156363 0.11483422 0.12402334 0.20355269]
[0.1106208  0.11421487 0.12348139 0.20388228]
[0.10903827 0.11329056 0.12251379 0.20506695]
[0.10861864 0.1129546  0.12211177 0.20540509]
[0.10802666 0.11251532 0.12180922 0.20359494]
[0.10809964 0.11251282 0.12184742 0.20328178]
[0.10789525 0.11236557 0.12185172 0.20181251]
[0.1078277  0.11238775 0.12180501 0.20202577]
[0.10830327 0.11284892 0.12225782 0.20118916]
[0.10843413 0.11267972 0.12215294 0.19965764]
[0.10836564 0.11236478 0.12190191 0.1976699 ]
[0.10763204 0.1118677  0.12150295 0.1969869

In [5]:
with rasterio.open("/data/DIGITANIE/Toulouse/Toulouse.tif") as f:
    print(f.statistics(1, approx=True))
    print(f.shape)

Statistics(min=0.0, max=1.1007251739502, mean=0.061889644308635, std=0.061194959199135)
(54062, 55600)


In [12]:
import rasterio
from rasterio.enums import Resampling
from rasterstats import zonal_stats

scale_factor = 1/10

with rasterio.open("/data/DIGITANIE/Toulouse/Toulouse.tif") as dataset:

    # resample data to target shape
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * scale_factor),
            int(dataset.width * scale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )
    
print(data.shape)
for band in range(1, 5):
    print(zonal_stats(
        DATA_POLYGON, 
        data[band-1],
        stats="count min mean max median",
        affine=transform,
        band_num=band
    ))

(4, 5406, 5560)
[{'min': 0.027740102261304855, 'max': 1.0329269170761108, 'mean': 0.10590577293649074, 'count': 14482283, 'median': 0.09561806917190552}]
[{'min': 0.04800046235322952, 'max': 0.9886831045150757, 'mean': 0.11046101122316143, 'count': 14482283, 'median': 0.10330308228731155}]
[{'min': 0.07036241143941879, 'max': 0.9419602751731873, 'mean': 0.11877787155519609, 'count': 14482283, 'median': 0.11214419454336166}]
[{'min': 0.01943548023700714, 'max': 1.0317635536193848, 'mean': 0.2077604235464809, 'count': 14482283, 'median': 0.21003715693950653}]


[{'min': 0.027740102261304855,
  'max': 1.0329269170761108,
  'mean': 0.10590577293649074,
  'count': 14482283,
  'median': 0.09561806917190552}]